In [1]:
import genet
from genet import analysis as ans
import os, sys, regex, glob, shutil, itertools, time, subprocess
import pandas as pd
import multiprocessing as mp
from Bio import SeqIO
from tqdm import tqdm
from genet.utils import *

In [2]:
class SortByBarcodes:
    '''# SortByBarcodes

    This class makes new fastq files only containing specific barcode sequences.
    The barcode list should be input files as DNA sequences.
    The barcode pattern is string based on regular expressions.

    #### Example
    >>> from genet import analysis as ans
    >>> ans.SortByBarcodes('./MyNGS.fastq', './Barcode_pattern.csv', 'TCGTATGCCGTCTTCTGCTTG[ATGC]{14}', n_cores=10)

    The output file will be generated in current working directory in default.
    If you want to save your output at other path, you can set the 'output_path' option.

    #### Example
    >>> ans.SortByBarcodes(seq_file='./MyNGS.fastq',
                           barcode_file='./Barcode_pattern.csv',
                           barcode_pattern='TCGTATGCCGTCTTCTGCTTG[ATGC]{14}', 
                           output_name='My_sorted_data',
                           output_path='/extdata/mydir/results',
                           n_cores=20
                           )
    '''
    
    def __init__(self,
                 seq_file:str,
                 barcode_file:str,
                 barcode_pattern:str = None,
                 output_name:str = 'barcode_sorted', 
                 output_path:str = './',
                 data_format:str = 'fastq',
                 output_format:str = 'fastq',
                 n_cores:int = int(mp.cpu_count()*0.5),
                 remove_temp_files:bool = True,
                 silence:bool = False,
                 ):

        # check input types
        if n_cores > mp.cpu_count():
            sys.exit('n_core should be lower than the number of cores which your machine has')
        
        # make temp file directory to save split fastq files
        self.sTEMP_DIR = '%s/%s_temp' % (output_path, output_name)
        os.makedirs(output_path, exist_ok=True)
        os.makedirs(self.sTEMP_DIR, exist_ok=True)

        # load barcode and data files
        # self.df_bc    = pd.read_csv(barcode_file, names=['id', 'barcode'])
        
        '''legacy code - 문제 없으면 삭제 예정
        self.records  = list(SeqIO.parse(open(seq_file), data_format))
        self.total    = len(self.records)
        
        # split fastq file
        list_nBins = [[int(self.total * (i + 0) / n_cores), int(self.total * (i + 1) / n_cores)] for i in range(n_cores)]
        self.list_sParameters = []
        
        for nStart, nEnd in list_nBins:
            if silence == False: print('[Info] Make data subsplits: %s - %s' % (nStart, nEnd))
            list_sSubSplits = self.records[nStart:nEnd]
            sSplit_fastq_DIR = '%s/idx_%s-%s' % (self.sTEMP_DIR, nStart, nEnd)
            os.makedirs(sSplit_fastq_DIR, exist_ok=True)
            SeqIO.write(list_sSubSplits, '%s/_subsplits.%s' % (sSplit_fastq_DIR, output_format), output_format)
            self.list_sParameters.append([self.df_bc, barcode_pattern, nStart, nEnd, sSplit_fastq_DIR, output_format, silence])
            del list_sSubSplits
        
        del self.records

        '''

        # now this functiona only available for fastq input file
        '''
        with open(seq_file, 'r') as f:
            lines   = f.readlines()
            total   = len(lines)
            lineset = 4
            rec_cnt = total / lineset

            list_nBins = [[int(rec_cnt * (i + 0) / n_cores), int(rec_cnt * (i + 1) / n_cores)] for i in range(n_cores)]
            
            for nStart, nEnd in list_nBins:
                if silence == False: print('[Info] Make data subsplits: %s - %s' % (nStart, nEnd))
                sSplit_fastq_DIR = '%s/idx_%s-%s' % (self.sTEMP_DIR, nStart, nEnd)
                os.makedirs(sSplit_fastq_DIR, exist_ok=True)

                sSplit_file_name = '%s/_subsplits.%s' % (sSplit_fastq_DIR, output_format)
                with open(sSplit_file_name, 'w') as outfile:
                    for l in lines[nStart*lineset:nEnd*lineset]: outfile.write(l)
        '''
        genet.utils.split_fastq(seq_file, n_cores, 
                                out_path=output_path, 
                                out_name=output_name, 
                                silence=silence)

In [3]:
SortByBarcodes('./test_sample.fastq', '')

[Info] Make data subsplits: 0 - 3263
[Info] Make data subsplits: 3263 - 6526
[Info] Make data subsplits: 6526 - 9790
[Info] Make data subsplits: 9790 - 13053
[Info] Make data subsplits: 13053 - 16316
[Info] Make data subsplits: 16316 - 19580
[Info] Make data subsplits: 19580 - 22843
[Info] Make data subsplits: 22843 - 26107
